<h1 style="color:rgb(0,120,170)">Artificial Intelligence in Life Sciences</h1>
<h2 style="color:rgb(0,120,170)">QSAR and model evaluation</h2>

<b>Authors:</b> Rumetshofer, Renz, Schimunek <br>
<b>Date:</b> 24-03-2022

This file is part of the "Artificial Intelligence in Life Sciences" lecture material.
The following copyright statement applies to all code within this file.

<b>Copyright statement:</b><br>
This material, no matter whether in printed or electronic form, may be used for personal and non-commercial educational
use only. Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed
or in electronic form, requires explicit prior acceptance of the authors.

In [5]:
!pip install rdkit
import os
import pandas as pd
import numpy as np
import copy

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

from rdkit import RDLogger  
RDLogger.DisableLog('rdApp.*') 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier




from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

print(rdkit.__version__)

<h1 style="color:rgb(0,120,170)">The Tox21 dataset</h1>

The Tox21 dataset comprises roughly 13,000 compounds (12,060 training samples, 647 test samples) tested for 12 different toxicological assays (active/inactive). The label matrix contains a lot of missing labels denoted as <i>NA</i>.

More information about the dataset can be found here: https://tripod.nih.gov/tox21/challenge/

Here we use a version of the dataset where the samples have been clustered and then assigned to 5 different folds.

In [2]:
# Preprocessed Tox21 dataset with pre-assigned clusters
data = pd.read_csv("../input/ails-first/data_train.csv",index_col=0).reset_index(drop=True)
data

<h2 style="color:rgb(0,120,170)">Data preprocessing</h2>

In order to use the dataset for training a model we replace the missing values with `-1`.

In [3]:
# Select labels, replace NaNs, convert to numpy array
y = data[data.columns[1:]].fillna(-1)
y = y.to_numpy()
y.shape

Next, we calculate Morgan fingerprints from the Smiles string for each sample.

In [4]:
# Initialize variables
fp_length = 1024
fps = np.zeros((len(data), fp_length))

# Calculate Morgan fingerprints and convert to numpy array
for i, smiles in enumerate(tqdm(data['smiles'])):
    mol = Chem.MolFromSmiles(smiles)
    fp_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=fp_length)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp_vec, arr)
    fps[i] = arr

In [5]:
#Test data Fps
test = pd.read_csv("../input/ails-first/smiles_test.csv")
# Initialize variables
fp_length = 1024
fps_test = np.zeros((len(test), fp_length))

# Calculate Morgan fingerprints and convert to numpy array
for i, smiles in enumerate(tqdm(test['smiles'])):
    mol = Chem.MolFromSmiles(smiles)
    fp_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=fp_length)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp_vec, arr)
    fps_test[i] = arr

<h1 style="color:rgb(0,120,170)">Train model on random split</h1>

We can now train a random forest model using the Morgan fingerprints as input. Here we don't use the clustering information but train the model on the standard train/test split where samples were randomly assigned to the respective set.

In [18]:
# Function returning the training and test sets
from imblearn.ensemble import BalancedBaggingClassifier

def split_data(test_fold, X, y, folds):
    test_mask = test_fold==folds.values
    X_test = X[test_mask]
    y_test = y[test_mask]

    train_mask = test_fold!=folds.values
    X_train = X[train_mask]
    y_train = y[train_mask]

    return X_train, X_test, y_train, y_test

# Train a random forest model for each task on the supplied training set and return predictions for the test set
def train_rf(X_train, y_train, X_test):
    seed = 1234
    n_tasks = y_train.shape[1]
    y_hats_proba = np.empty((X_test.shape[0], n_tasks))
    y_hats_class = np.empty_like(y_hats_proba)
    
    # Train RF per task
    for j in tqdm(range(n_tasks)):
        rf_model = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators = 500, max_features = 'auto', max_depth = 8, criterion = 'gini'),
                                sampling_strategy='not majority',
                                replacement=False,
                                random_state=42)
       # Mask out unknown samples
        idx = (y_train[:, j] != (0))
        # Train model
        rf_model.fit(X_train[idx], y_train[idx, j])
        # Predict class probabilities (select only values for positiv class with index 1)
        y_hats_proba[:, j] = rf_model.predict_proba(X_test)[:, 1]
        # Predict class 
        y_hats_class[:, j] = rf_model.predict(X_test)
    return y_hats_proba, y_hats_class 

In [19]:
# Split data into training and test set using the random split from the dataset
X_train, X_test, y_train, y_test = train_test_split(fps, y ,test_size=0.2)
# Train a random forest model and get predictions for the test set
y_hats_proba, y_hats_class = train_rf(X_train, y_train, fps_test)

We can look at the predicted probabilities and classes. Since the model doesn't know which values of the test set are actually measured and which are missing we get a prediction for each sample.

In [20]:
result = pd.DataFrame(y_hats_proba)
result.columns =  ['task1','task2','task3','task4','task5','task6','task7','task8','task9','task10','task11']
result.to_csv('result.csv')

In [10]:
pd.DataFrame(y_hats_class)

We can also look at the predicted number of samples for each class.

In [11]:
unique, counts = np.unique(y_hats_class, return_counts=True)
dict(zip(unique, counts))

<h1 style="color:rgb(0,120,170)">Metrics</h1>

To determine the quality of the model we look at several metrics. When calculating metrics we need to remove predictions for missing values as there's no way to measure the quality of these predictions.

<h2 style="color:rgb(0,120,170)">Confusion Matrix, Precision, Recall, F1-score</h2>

Lets look at these metrics (or methods) for the first task.

In [12]:
task = 0
# Mask out unknown samples
idx = (y_test[:, task] != (-1))

### Confusion Matrix

In [13]:
cm = confusion_matrix(y_test[idx,task], y_hats_class[idx,task])
cm

In [14]:
# True Negatives, False Positives, False Negatives, True Positives
cm.ravel()

### Precision, Recall and F1-Score

- The **precision** is the ratio $\frac{TP}{TP + FP}$ where TP is the number of true positives and FP the number of false positives. The precision is intuitively the ability of the classifier to not label negative samples as positive.

- The **recall** is the ratio $\frac{TP}{TP + FN}$ where TP is the number of true positives and FN the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

- The **F1-score** can be interpreted as a weighted harmonic mean of the precision and recall.

In [15]:
print(classification_report(y_test[idx,task],y_hats_class[idx,task], target_names=["class 0", "class 1"]))

<h2 style="color:rgb(0,120,170)">Area under the ROC curve (AUC)</h2>

Next, we calculate the AUC for each task and the mean over all tasks.

In [16]:
def calc_masked_AUC_per_task(prediction, target):
    auc_per_task = []
    for j in range(target.shape[1]):
        y_score = prediction[:, j]
        y_true = target[:, j]
        # Mask out unknown samples
        idx = (y_true != (-1))
        # Calculate AUC per task
        auc_per_task.append(roc_auc_score(y_true[idx], y_score[idx]))
    return auc_per_task

In [17]:
# Calculate AUC per task
auc_per_task = calc_masked_AUC_per_task(y_hats_proba, y_test)
auc_per_task

In [ ]:
np.mean(auc_per_task)

<h1 style="color:rgb(0,120,170)">Cluster Cross-Validation</h1>

The previous model was trained with samples randomly assigned to the training and test sets. However, if we want to know how well our model generalizes to future data it might be a better idea to assign the training and test samples based on structural similarity. If we cluster the samples and assign all samples of some clusters to the training set and all samples of the other clusters to the test set we avoid that very similar samples are in the training and test sets.

In [7]:
first = pd.read_csv("../input/result-ails-final/result_gamble.csv")
first = first[['task1','task3','task4','task5','task9','task10','task11']]
second = pd.read_csv("../input/result-ails-final/2nd_best.csv")
second = second[['task2','task7','task8']]
third = pd.read_csv("../input/result-ails-final/task6.csv")
third = third[['task6']]


result = pd.concat([first, second,third], axis=1)
result =  result[['task1','task2','task3','task4','task5','task6','task7','task8','task9','task10','task11']]
print(result)

result.to_csv('result_final.csv')